In [3]:
# execute this in command line on all machines to be used as workers before initiating the hyperparamer search 
# ! pip install -U trains-agent==0.15.0
# ! trains-agent daemon --queue default

# pip install with locked versions
# ! pip install -U pandas>=1.0.3
# ! pip install -U trains>=0.16.2
# ! pip install -U optuna==2.0.0

In [4]:
from trains.automation import UniformParameterRange, UniformIntegerParameterRange
from trains.automation import HyperParameterOptimizer
from trains.automation.optuna import OptimizerOptuna

from trains import Task

In [5]:
task = Task.init(project_name='Hyperparameter Optimization with Optuna',
                 task_name='Hyperparameter Search',
                 task_type=Task.TaskTypes.optimizer)


TRAINS Task: overwriting (reusing) task id=adfb92750a4545b5a73ef857e123f38a
2020-10-07 01:00:26,026 - trains.Repository Detection - WARNING - Can't get branch information for git repo in /home/tc/Programming/Python/Miniconda3/envs/ml-training/lib/python3.8/site-packages
2020-10-07 01:00:26,034 - trains.Repository Detection - WARNING - Can't get commit information for git repo in /home/tc/Programming/Python/Miniconda3/envs/ml-training/lib/python3.8/site-packages
2020-10-07 01:00:26,046 - trains.Repository Detection - WARNING - Can't get root information for git repo in /home/tc/Programming/Python/Miniconda3/envs/ml-training/lib/python3.8/site-packages
2020-10-07 01:00:26,055 - trains.Repository Detection - WARNING - Can't get status information for git repo in /home/tc/Programming/Python/Miniconda3/envs/ml-training/lib/python3.8/site-packages
2020-10-07 01:00:26,074 - trains.Repository Detection - WARNING - Can't get diff information for git repo in /home/tc/Programming/Python/Miniconda

In [7]:
#####################################################################
### Don't forget to replace this default id with your own task id ###
#####################################################################
TEMPLATE_TASK_ID = 'adfb92750a4545b5a73ef857e123f38a'

In [10]:
optimizer = HyperParameterOptimizer(
    base_task_id=TEMPLATE_TASK_ID,  # This is the experiment we want to optimize
    # here we define the hyper-parameters to optimize
    hyper_parameters=[
        UniformIntegerParameterRange('number_of_epochs', min_value=2, max_value=12, step_size=2),
        UniformIntegerParameterRange('batch_size', min_value=2, max_value=16, step_size=2),
        UniformParameterRange('dropout', min_value=0, max_value=0.5, step_size=0.05),
        UniformParameterRange('base_lr', min_value=0.00025, max_value=0.01, step_size=0.00025),
    ],
    # setting the objective metric we want to maximize/minimize
    objective_metric_title='accuracy',
    objective_metric_series='total',
    objective_metric_sign='max',  # maximize or minimize the objective metric

    # setting optimizer - trains supports GridSearch, RandomSearch, OptimizerBOHB and OptimizerOptuna
    optimizer_class=OptimizerOptuna,
    
    # Configuring optimization parameters
    execution_queue='default',  # queue to schedule the experiments for execution
    max_number_of_concurrent_tasks=2,  # number of concurrent experiments
    optimization_time_limit=60.,  # set the time limit for the optimization process
    compute_time_limit=120,  # set the compute time limit (sum of execution time on all machines)
    total_max_jobs=20,  # set the maximum number of experiments for the optimization. 
                        # Converted to total number of iteration for OptimizerBOHB
    min_iteration_per_job=15000,  # minimum number of iterations per experiment, till early stopping
    max_iteration_per_job=150000,  # maximum number of iterations per experiment
)

2020-10-07 01:02:53,033 - trains.automation.optimization - WARNING - Could not find requested hyper-parameters ['number_of_epochs', 'batch_size', 'dropout', 'base_lr'] on base task adfb92750a4545b5a73ef857e123f38a
2020-10-07 01:02:53,300 - trains.automation.optimization - WARNING - Could not find requested metric ('accuracy', 'total') report on base task adfb92750a4545b5a73ef857e123f38a


In [ ]:
optimizer.set_report_period(1)  # setting the time gap between two consecutive reports
optimizer.start()  
optimizer.wait()  # wait until process is done
optimizer.stop()  # make sure background optimization stopped

Progress report #0 completed, sleeping for 0.25 minutes
2020-10-07 01:02:55,541 - trains.automation.optimization - INFO - Creating new Task: {'number_of_epochs': 4, 'batch_size': 2, 'dropout': 0.0, 'base_lr': 0.00125}
2020-10-07 01:02:56,008 - trains.automation.optimization - INFO - Creating new Task: {'number_of_epochs': 4, 'batch_size': 12, 'dropout': 0.4, 'base_lr': 0.00575}
Progress report #1 completed, sleeping for 1.0 minutes
Progress report #3 completed, sleeping for 1.0 minutes
Progress report #2 completed, sleeping for 1.0 minutes
Progress report #4 completed, sleeping for 1.0 minutes
Progress report #3 completed, sleeping for 1.0 minutes
Progress report #5 completed, sleeping for 1.0 minutes
Progress report #4 completed, sleeping for 1.0 minutes
Progress report #6 completed, sleeping for 1.0 minutes
OptunaObjective result metric=None, iteration None


[W 2020-10-07 01:07:00,446] Trial 1 failed, because the returned value from the objective function cannot be cast to float. Returned value is: None


OptunaObjective result metric=None, iteration None


[W 2020-10-07 01:07:00,756] Trial 0 failed, because the returned value from the objective function cannot be cast to float. Returned value is: None


2020-10-07 01:07:02,466 - trains.automation.optimization - INFO - Creating new Task: {'number_of_epochs': 4, 'batch_size': 16, 'dropout': 0.25, 'base_lr': 0.00825}
2020-10-07 01:07:02,745 - trains.automation.optimization - INFO - Creating new Task: {'number_of_epochs': 12, 'batch_size': 12, 'dropout': 0.2, 'base_lr': 0.0035000000000000005}
2020-10-07 01:07:14,779 - trains.Task - WARNING - ### TASK STOPPED - USER ABORTED - STATUS CHANGED ###
Progress report #5 completed, sleeping for 1.0 minutes


In [ ]:
# optimization is completed, print the top performing experiments id
k = 3
top_exp = optimizer.get_top_experiments(top_k=k)
print('Top {} experiments are:'.format(k))
for n, t in enumerate(top_exp, 1):
    print('Rank {}: task id={} |result={}'
          .format(n, t.id, t.get_last_scalar_metrics()['accuracy']['total']['last']))